## Install Cobrapy Package

In [0]:
!pip install cobra

# Simulating with FBA

Simulations using flux balance analysis can be solved using `Model.optimize()`. This will maximize or minimize (maximizing is the default) flux through the objective reactions.

In [0]:
import cobra.test
model = cobra.test.create_test_model("textbook")

## Running FBA

Get the objective of a model

In [0]:
print(model.objective)

In [0]:
biomass_rxn = model.reactions.get_by_id('Biomass_Ecoli_core')
biomass_rxn

In [0]:
biomass_rxn.id = 'BIOMASS_Ecoli_core_w_GAM'

In [0]:
solution = model.optimize()
print(solution)

The Model.optimize() function will return a Solution object. A solution object has several attributes:

 - `objective_value`: the objective value
 - `status`: the status from the linear programming solver
 - `fluxes`: a pandas series with flux indexed by reaction identifier. The flux for a reaction variable is the difference of the primal values for the forward and reverse reaction variables. (More about [Pandas Series and Dataframes](https://pandas.pydata.org))
 - `shadow_prices`: a pandas series with shadow price indexed by the metabolite identifier.

For example, after the last call to `model.optimize()`, if the optimization succeeds it's status will be optimal. In case the model is infeasible an error is raised.

In [0]:
solution.objective_value

Get flux distribution of the current soultion:

In [0]:
solution.fluxes

### Analyzing FBA solutions

Models solved using FBA can be further analyzed by using summary methods, which output printed text to give a quick representation of model behavior. Calling the summary method on the entire model displays information on the input and output behavior of the model, along with the optimized objective.

In [0]:
model.summary()

In addition, the input-output behavior of individual metabolites can also be inspected using summary methods. For instance, the following commands can be used to examine the overall redox balance of the model

In [0]:
model.metabolites.nadh_c.summary()

Or to get a sense of the main energy production and consumption reactions

In [0]:
model.metabolites.atp_c.summary()

## Visualizing FBA solutions with Escher Maps



Create a directory for results and save flux solution in a json file.

In [0]:
import os
from google.colab import files

directory = "results"
if not os.path.exists(directory):
    os.makedirs(directory)
       
solution.fluxes.to_json('results/flux_1.json')
files.download('results/flux_1.json')

To view the flux solution on a map:



1.   Go to [Escher Maps](https://escher.github.io)
2.   Choose *Escherichia Coli* as organism, *Core metabolism (e_coli_core)* as map, *e_coli_core* as model and *Viewer* as Tool
3.   Press *Load map*
4.  You will be redirected to the metabolic map of E.coli
5. To map your flux solution, go to the *Menu > Data > Load reaction data* and choose the* json file*, you just downloaded
6. Check the visualisation to your likings, go to *Menu > View > Settings*




## Changing the Objectives

The objective function is determined from the objective_coefficient attribute of the objective reaction(s). Generally, a "biomass" function which describes the composition of metabolites which make up a cell is used.

In [0]:
biomass_rxn

The objective function can be changed by assigning Model.objective, which can be a reaction object (or just it's name), or a `dict` of `{Reaction: objective_coefficient}`.

In [0]:
ATPM_rxn = model.reactions.get_by_id("ATPM")
ATPM_rxn

In [0]:
# change the objective to ATPM
model.objective = "ATPM"

In [0]:
solution = model.optimize()
solution.objective_value

In [0]:
solution.fluxes.to_json('results/flux_2.json')
files.download('results/flux_2.json')


Go to again to [Escher Maps](https://escher.github.io) and load the flux solution.

So far, we always searched for the maximum:

In [0]:
model.objective_direction

but as for maintenace in terms of ATP minimisation might make more sense. You can simple set it like this:

In [0]:
model.objective_direction = 'min'
model.objective_direction

In [0]:
solution = model.optimize()
solution.objective_value

In [0]:
solution.fluxes.to_json('results/flux_3.json')
files.download('results/flux_3.json')

Go to again to [Escher Maps](https://escher.github.io) and load the flux solution.

## Running FVA

FBA will not give always give unique solution, because multiple flux states can achieve the same optimum. FVA (or flux variability analysis) finds the ranges of each metabolic flux at the optimum.

In [0]:
model.objective_direction = 'max'

In [0]:
from cobra.flux_analysis import flux_variability_analysis

In [0]:
flux_variability_analysis(model, model.reactions[:10])

Setting parameter `fraction_of_optimium=0.90` would give the flux ranges for reactions at 90% optimality.

In [0]:
cobra.flux_analysis.flux_variability_analysis(
    model, model.reactions[:10], fraction_of_optimum=0.9)

The standard FVA may contain loops, i.e. high absolute flux values that only can be high if they are allowed to participate in loops (a mathematical artifact that cannot happen in vivo). Use the `loopless` argument to avoid such loops. Below, we can see that FRD7 and SUCDi reactions can participate in loops but that this is avoided when using the looplesss FVA.

In [0]:
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=False)

In [0]:
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=True)

### Running FVA in summary methods

Flux variability analysis can also be embedded in calls to summary methods. For instance, the expected variability in substrate consumption and product formation can be quickly found by

In [0]:
model.optimize()
model.summary(fva=0.95)

Similarly, variability in metabolite mass balances can also be checked with flux variability analysis.

In [0]:
model.metabolites.pyr_c.summary(fva=0.95)

In these summary methods, the values are reported as a the center point +/- the range of the FVA solution, calculated from the maximum and minimum values.

## Running pFBA

Parsimonious FBA (often written pFBA) finds a flux distribution which gives the optimal growth rate, but minimizes the total sum of flux. This involves solving two sequential linear programs, but is handled transparently by cobrapy. For more details on pFBA, please see [Lewis et al. (2010)](http://dx.doi.org/10.1038/msb.2010.47).

For small models like the *textbook* model that we have used so far, the difference might not be extensive, because the amount of alternative solutions is also limited. Therefore, this time we load the *ecoli* test model, it is larger than the *textbook* model.

In [0]:
model = cobra.test.create_test_model("ecoli")
model

It also comes with a another biomass function *Ec_biomass_iJO1366_WT_53p95M* that we can use. Let's use this one to practice coding:

In [0]:
model.objective = 'Ec_biomass_iJO1366_WT_53p95M'

In [0]:
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)

These functions should give approximately the same objective value.

In [0]:
abs(fba_solution.fluxes["Ec_biomass_iJO1366_WT_53p95M"] - pfba_solution.fluxes[
    "Ec_biomass_iJO1366_WT_53p95M"])

Compare the total sum of fluxes



In [0]:
pfba_sum = sum([abs(rxn) for rxn in pfba_solution.fluxes])
fba_sum = sum([abs(rxn) for rxn in fba_solution.fluxes])

print('pFBA: %s' %pfba_sum)
print('FBA: %s' %fba_sum)

Export flux solutions:

In [0]:
pfba_solution.fluxes.to_json('results/flux_4.json')
fba_solution.fluxes.to_json('results/flux_5.json')
files.download('results/flux_4.json')
files.download('results/flux_5.json')

Go again to [Escher Maps](https://escher.github.io) and load the flux solutions but this time use the *Central Metabolism* map.